In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [12]:
def generate_key_for_evaluation(df):
    left_table_name = []
    left_column_name = []
    right_table_name = []
    right_column_name = []
    df_key = pd.DataFrame((), columns=df.columns)
    
    for i in range(df.shape[0]):
        rtn = df.iloc[i, 2]
        ltn = df.iloc[i, 0]
        right_table_name.append(rtn if rtn > ltn else ltn)
        left_table_name.append(rtn if rtn < ltn else ltn)
    
        rcn = df.iloc[i, 3]
        lcn = df.iloc[i, 1]
        right_column_name.append(rcn if rtn > ltn else lcn)
        left_column_name.append(rcn if rtn < ltn else lcn)
    
    d = {'LEFT_TABLE':left_table_name,
         'LEFT_COLUMN':left_column_name,
         'RIGHT_TABLE':right_table_name,
         'RIGHT_COLUMN':right_column_name}
    df_key = pd.DataFrame(d)
    df_key = df_key.drop_duplicates()
    df_key = df_key.reset_index(drop=True)
    
    key = []
    for i in range(df_key.shape[0]):
        key.append("#".join(df_key.iloc[i,:]))
    df_key['KEY'] = key

    return df_key

In [11]:
cta_matches = pd.read_csv('Description_test/CTA_from_descriptions/JD_from_cta_matches.csv')
cta_matches = cta_matches[cta_matches['JOINABLE'] == 'Yes'][['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN']]
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN
69,statewise-census-data-in-india-1901-2011.csv,DATE,population-by-governorate-citizenship-and-gend...,Year
74,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,currency_exchange.csv,Country
214,road-ahead-current-road-closures.csv,PROJECT,road-ahead-upcoming-projects.csv,PROJECT
216,road-ahead-current-road-closures.csv,PROJECT,road-ahead-upcoming-projects.csv,LOCATION
224,road-ahead-current-road-closures.csv,PROJECT,road-ahead-projects-under-construction.csv,PROJECT


In [13]:
cta_matches = generate_key_for_evaluation(cta_matches)
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,population-by-governorate-citizenship-and-gend...,Year,statewise-census-data-in-india-1901-2011.csv,DATE,population-by-governorate-citizenship-and-gend...
1,currency_exchange.csv,Country,statewise-census-data-in-india-1901-2011.csv,LOCATION_NAME,currency_exchange.csv#Country#statewise-census...
2,road-ahead-current-road-closures.csv,PROJECT,road-ahead-upcoming-projects.csv,PROJECT,road-ahead-current-road-closures.csv#PROJECT#r...
3,road-ahead-current-road-closures.csv,PROJECT,road-ahead-upcoming-projects.csv,LOCATION,road-ahead-current-road-closures.csv#PROJECT#r...
4,road-ahead-current-road-closures.csv,PROJECT,road-ahead-projects-under-construction.csv,PROJECT,road-ahead-current-road-closures.csv#PROJECT#r...


In [15]:
cta_matches.shape

(250, 5)

In [14]:
groundTruth = pd.read_csv('joinable_columns_gt3_quality.csv')
groundTruth = groundTruth[['ds_name', 'att_name', 'ds_name_2', 'att_name_2']]
groundTruth = generate_key_for_evaluation(groundTruth)
groundTruth.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,road-ahead-current-road-closures.csv,COMP_DATE,road-ahead-upcoming-projects.csv,COMP_DATE,road-ahead-current-road-closures.csv#COMP_DATE...
1,eo_pr.csv,NTEE_CD,eo_xx.csv,NTEE_CD,eo_pr.csv#NTEE_CD#eo_xx.csv#NTEE_CD
2,eo4.csv,NAME,eo_pr.csv,NAME,eo4.csv#NAME#eo_pr.csv#NAME
3,eo4.csv,ICO,eo_pr.csv,ICO,eo4.csv#ICO#eo_pr.csv#ICO
4,eo4.csv,STREET,eo_pr.csv,STREET,eo4.csv#STREET#eo_pr.csv#STREET


In [16]:
groundTruth.shape

(53, 5)

In [17]:
tp = 0
fp = 0
fn = 0

for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] in groundTruth.iloc[:,4].values):
        tp += 1
    else:
        fp += 1
    
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        fn += 1
    
precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2 * (precision*recall)/(precision+recall)

print(precision, recall, f1_score)

0.196 0.9245283018867925 0.32343234323432346


# Falso Positivo

In [19]:
for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] not in groundTruth.iloc[:,4].values):
        print(cta_matches.iloc[i,4])

population-by-governorate-citizenship-and-gender.csv#Year#statewise-census-data-in-india-1901-2011.csv#DATE
currency_exchange.csv#Country#statewise-census-data-in-india-1901-2011.csv#LOCATION_NAME
road-ahead-current-road-closures.csv#PROJECT#road-ahead-upcoming-projects.csv#PROJECT
road-ahead-current-road-closures.csv#PROJECT#road-ahead-upcoming-projects.csv#LOCATION
road-ahead-current-road-closures.csv#PROJECT#road-ahead-projects-under-construction.csv#PROJECT
road-ahead-current-road-closures.csv#PROJECT#road-ahead-projects-under-construction.csv#STREET
road-ahead-current-road-closures.csv#PROJECT#road-ahead-projects-under-construction.csv#LOCATION
road-ahead-current-road-closures.csv#STREET#road-ahead-upcoming-projects.csv#STREET
road-ahead-current-road-closures.csv#STREET#road-ahead-projects-under-construction.csv#PROJECT
road-ahead-current-road-closures.csv#STREET#road-ahead-projects-under-construction.csv#STREET
road-ahead-current-road-closures.csv#STREET#road-ahead-projects-under

# Falso Negativo

In [18]:
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        print(groundTruth.iloc[i,4])

eo4.csv#CITY#eo_pr.csv#CITY
population-census-of-botswana-2011.csv#FREQUENCY#statewise-census-data-in-india-1901-2011.csv#FREQUENCY
eo4.csv#ICO#eo_xx.csv#ICO
eo4.csv#CITY#eo_xx.csv#CITY
